## Simulación con distribución empírica (Camiones)

In [1]:
from sympy import *
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from IPython.display import display, Math
import random
import math
import pandas as pd

start = 10
end = 30
bins = 5
heights = [14, 18, 9, 17, 22]

int_dif = (end-start)/bins

diff = Rational((end-start), bins)

#Lista de vertices
vertices = []
actual = start
vertices.append([start, 0])
actual += int_dif/2

for i in range(1, bins+1):
    vertices.append([actual, heights[i-1]])
    actual += int_dif


vertices.append([actual-int_dif/2, 0])

n = len(vertices)

# Apply the Shoelace formula:
# A = 0.5 * |(x1*y2 + x2*y3 + ... + xn*y1) - (y1*x2 + y2*x3 + ... + yn*x1)|
area = 0.0
for i in range(n):
    j = (i + 1) % n  # The next vertex, wrapping around from the last to the first
    
    # Sum of x_i * y_{i+1}
    area += vertices[i][0] * vertices[j][1]
    
    # Subtraction of y_i * x_{i+1}
    area -= vertices[j][0] * vertices[i][1]

final_area = abs(area) / 2.0

print('Area:', final_area)

Area: 284.0


In [2]:
#Ecuaciones que describen el histograma sin escalar
y = Symbol('y')
x = Symbol('x')

f = []
f_scaled = []

m1 = Rational(heights[0] , (Rational(diff, 2)))

f.append(m1*x - m1*start)
f_scaled.append(Rational(m1, final_area)*x - Rational(m1*start, final_area))

actual = start + int_dif/2
for i in range(1, bins):
    m = Rational(heights[i]-heights[i-1] , int_dif)
    f.append(m*x - m*Integer(actual) + heights[i-1])
    f_scaled.append(Rational(m, final_area)*x - Rational(m*Integer(actual), final_area) + Rational(heights[i-1], final_area))
    actual +=int_dif

m1 = Rational(heights[len(heights)-1], -int_dif/2)
f.append(m1*x -m1*Integer(actual)+heights[len(heights)-1])
f_scaled.append(Rational(m1, final_area)*x - Rational(m1*Integer(actual), final_area) +Rational(heights[len(heights)-1], final_area))


print('----------------------------------------- \nEcuaciones obtenidas que describen el histograma sin escalar: \n ----------------------------------------')

actual = start

# Primera ecuación
display(Math(rf"(1): \; {latex(f[0])}, \; {actual} \leq x \leq {actual + int_dif/2}"))

# Ecuaciones intermedias
actual += int_dif/2
for i in range(1, bins):
    display(Math(rf"({i+1}): \; {latex(f[i])}, \; {actual} \leq x \leq {actual + int_dif}"))
    actual += int_dif

# Última ecuación
display(Math(rf"({bins+1}): \; {latex(f[bins])}, \; {actual} \leq x \leq {actual + int_dif/2}"))

----------------------------------------- 
Ecuaciones obtenidas que describen el histograma sin escalar: 
 ----------------------------------------


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [3]:
print('----------------------------------------- \nEcuaciones obtenidas que describen el histograma escalado con area 1: \n ----------------------------------------')

actual = start

ranges = []

# Primera ecuación
display(Math(rf"(1): \; {latex(f_scaled[0])}, \; {actual} \leq x \leq {actual + int_dif/2}"))
ranges.append([actual, actual + int_dif/2])
# Ecuaciones intermedias
actual += int_dif/2
for i in range(1, bins):
    display(Math(rf"({i+1}): \; {latex(f_scaled[i])}, \; {actual} \leq x \leq {actual + int_dif}"))
    ranges.append([actual, actual + int_dif])
    actual += int_dif

# Última ecuación
display(Math(rf"({bins+1}): \; {latex(f_scaled[bins])}, \; {actual} \leq x \leq {actual + int_dif/2}"))
ranges.append([actual, actual + int_dif/2])

----------------------------------------- 
Ecuaciones obtenidas que describen el histograma escalado con area 1: 
 ----------------------------------------


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [4]:
x_vals, y_vals = [], []

actual = start
exprs = []
intervals = []

# First interval
exprs.append(f[0])
intervals.append((start, start + int_dif/2))

# Middle intervals
actual = start + int_dif/2
for i in range(1, bins):
    exprs.append(f[i])
    intervals.append((actual, actual+int_dif))
    actual += int_dif

# Last interval
exprs.append(f[bins])
intervals.append((actual, actual+int_dif/2))

# Sample each piecewise function
for expr, (a, b) in zip(exprs, intervals):
    func = lambdify(x, expr, 'numpy')
    xs = np.linspace(a, b, 50)
    ys = func(xs)
    x_vals.extend(xs)
    y_vals.extend(ys)

# Plot
fig = go.Figure()

bin_edges = np.linspace(start, end, bins+1)

fig.add_trace(go.Bar(
    x=(bin_edges[:-1] + bin_edges[1:]) / 2,  # centers for positioning
    y=heights,
    width=int_dif,
    opacity=0.4,
    name="Histogram"
))

# Piecewise line
fig.add_trace(go.Scatter(
    x=x_vals,
    y=y_vals,
    mode="lines",
    name="Piecewise Function",
    line=dict(color="red")
))

fig.update_layout(
    title="Histogram with Piecewise Linear Function",
    xaxis_title="x",
    yaxis_title="y",
    bargap=0
)

fig.show()

In [5]:
x, n, t, y = symbols('x n t y')

integrals = []

print('----------------------------------------- \n Distribución acumulada F(x): \n ----------------------------------------')

for i in range(bins+1):
    actual_func = f_scaled[i]
    ac_range = ranges[i]
    grl_integral = integrate(actual_func, (x, n, t))
    ev_integral = grl_integral.subs(n, int(ac_range[0]))
    #display(ev_integral)
    if(i != 0):
        #Adding the value of the last integral
        ev_integral = integrals[i-1].subs(t, int(ac_range[0])) + (ev_integral)

    integrals.append(factor(ev_integral))
    

    display(Math(rf"({i+1}) \; {latex(integrals[i])}, \; {ac_range[0]} \leq t \leq {ac_range[1]}"))


----------------------------------------- 
 Distribución acumulada F(x): 
 ----------------------------------------


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [6]:
inverses = []
invereses_ranges = []
print("----------------------------------------- \n F'(x): \n ----------------------------------------")

for i in range(bins+1):
    inv_1 = solve(integrals[i] - y, t)[0]
    inv_2 = solve(integrals[i] - y, t)[1]
    ac_range = ranges[i]
    invereses_ranges.append([integrals[i].subs(t, int(ac_range[0])), integrals[i].subs(t, int(ac_range[1]))])
    randi = random.uniform(float(invereses_ranges[i][0]), float(invereses_ranges[i][1]))
    #inverses.append(inv_2)
    #print(randi)
    #print(inv_1.subs(t, randi))
    if(float(inv_1.subs(y, randi)) >= float(ranges[i][0]) and float(inv_1.subs(y, randi)) <= float(ranges[i][1])):
        inverses.append(inv_1)
    elif(float(inv_2.subs(y, randi)) >= float(ranges[i][0]) and float(inv_2.subs(y, randi)) <= float(ranges[i][1])):
        inverses.append(inv_2)
    else:
        print(f'Not valid value in {i}')
        break
    display(Math(rf"({i+1}) \; {latex(inverses[i])}, \; {invereses_ranges[i][0]} \leq y \leq {invereses_ranges[i][1]}"))

----------------------------------------- 
 F'(x): 
 ----------------------------------------


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [7]:
print(invereses_ranges)

[[0, 7/142], [7/142, 39/142], [39/142, 33/71], [33/71, 46/71], [46/71, 131/142], [131/142, 1]]


# Simulaciones - Empírica y exponencial

In [8]:
time = 0
end_time = 14*60

def exp_inv(randi: float,l: float):
    return (-1/l)*math.log(1-randi)

In [9]:
days = 30
excel_file = "times2.xlsx"

for day in range(days):
    #Bus simulation
    next_bus_arrival_time = []
    bus_arrival_time = []
    sum = 0

    while(sum < end_time):
        randi = random.uniform(0, 1)
        for i in range(len(inverses)):
            if(randi>=invereses_ranges[i][0] and randi <= invereses_ranges[i][1]):
                val = float(inverses[i].subs(y, randi))
                #Last value still counts as the last bus leaves after 22:00 hrs
                next_bus_arrival_time.append(val)
                sum += val
                bus_arrival_time.append(sum)

    #Pax simulation
    pax_next_arrival_time = []
    pax_arrival_time = []
    time = 0
    arrival_time = 0

    while(time<end_time):
        randi = random.uniform(0, 1)
        if(time >= 0 and time <= 2*60):
            #first interval
            arrival_time = exp_inv(randi, 1/2)
        elif(time > 2*60 and time <= 5*60):
            #second interval
            arrival_time = exp_inv(randi, 1/3)
        elif(time> 5*60 and time <= 9*60):
            #third interval
            arrival_time = exp_inv(randi, 1/1.6)
        elif(time> 9*60 and time <= 14*60):
            #fourth interval
            arrival_time = exp_inv(randi, 1/2.6)
        
        #Assuming that no passengers can arrive after 22:00
        time += arrival_time
        if(time<end_time):
            pax_next_arrival_time.append(arrival_time)
            pax_arrival_time.append(time)

    #Writing in excel
    max_len = max(len(pax_arrival_time), len(bus_arrival_time))

    pax_arrival_time += [None] * (max_len - len(pax_arrival_time))
    bus_arrival_time += [None] * (max_len - len(bus_arrival_time))

    df = pd.DataFrame({
        "pax_arrival_time": pax_arrival_time,
        "bus_arrival_time ": bus_arrival_time
    })

    sheet_name = f"Day_{day+1}" 
    if(day == 0):
        with pd.ExcelWriter(excel_file, engine='openpyxl', mode='w') as writer:
            df.to_excel(writer, index=False, sheet_name=sheet_name)
    else:
        with pd.ExcelWriter(excel_file, engine='openpyxl', mode='a') as writer:
            df.to_excel(writer, index=False, sheet_name=sheet_name)